# Deep Session 7주차
## ResNet

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.8.1  Device: cuda


In [3]:
BATCH_SIZE = 32
EPOCHS = 10

In [4]:
train_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                  train = True,
                                  download = True,
                                  transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

test_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                train = False,
                                transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)


Extracting ../data/CIFAR_10\cifar-10-python.tar.gz to ../data/CIFAR_10


In [5]:
#반복적으로 이용하는 block 정의
class BasicBlock(nn.Module): #nn.Module 상속받아 정의
    def __init__(self, in_planes, planes, stride = 1): #in_planes는 input으로 이용되는 데이터의 채널 수, planes는 필터의 갯수
        super(BasicBlock, self).__init__() #nn.Module의 메서드를 상속받습니다.
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size = 3, stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(planes) #input의 분포가 달라짐에 따라 학습속도가 느려지는 것을 막기 위해 사용
        self.conv2 = nn.Conv2d(planes, planes, kernel_size = 3, stride = 1, padding = 1, bias = False) #위에서 통과한 결과값 사용
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.shortcut = nn.Sequential() #ResNet의 shortcut 정의
        if stride != 1 or in_planes != planes: #두번째 블럭부터 적용되는 shorcut
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size = 1, stride = stride, bias = False),
                nn.BatchNorm2d(planes))
    
    def forward(self, x): #계산과정
        out = F.relu(self.bn1(self.conv1(x))) #conv연산과 Batch Nomarlization 연산을 진행하고 ReLU를 적용
        out = self.bn2(self.conv2(out)) #위에서 나온 값에 두번째 conv, BN을 적용
        out += self.shortcut(x) #결과값과 shorcut을 통과한 결과값을 더함
        out = F.relu(out) #shorcut까지 적용한 out에 relu적용
        return out
        
class ResNet(nn.Module):
    def __init__(self, num_classes = 10): #클래스 갯수 10개로 고정
        super(ResNet, self).__init__()
        self.in_planes = 16 #in_planes값을 16으로 고정
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding = 1, bias = False) #Basic Block의 conv1과 다름
        #input으로 이용하는 컬러 이미지에 적용하는 필터
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride = 1)
        self.layer2 = self._make_layer(32, 2, stride = 2)
        self.layer3 = self._make_layer(64, 2, stride = 2)
        self.linear = nn.Linear(64, num_classes)
        
    def _make_layer(self, planes, num_blocks, stride): #여러 층의 레이어를 구성해 반환
        strides = [stride] + [1] * (num_blocks  - 1) #stride 범위를 BasicBlock마다 설정할 수 있도록 정의
        layers = []
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride)) #BasicBlock의 결과값을 추가
            self.in_planes = planes #in_planes를 plane로 업데이트(shorcut을 계산하기 위해)
        return nn.Sequential(*layers) #여러 층으로 생성한 레이어를 Sequential로 정의해 반환
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out) #input 16채널, output 16채널인 BasicBlock 2개
        out = self.layer2(out) #input 16채널, output 32채널인 BasicBlock 1개 #input 32채널, output 32채널인 BasicBlock 1개
        out = self.layer3(out) #input 32채널, output 64채널인 BasicBlock 1개 #input 64채널, output 64채널인 BasicBlock 1개
        out = F.avg_pool2d(out, 8) #다운샘플링
        out = out.view(out.size(0), -1) #일차원 벡터로 펼치기
        out = self.linear(out) #10개의 노드로 구성된 FC와 연결해 10 크기의 벡터 출력 #CIFAR-10 의 원핫 인코딩과 비교해 loss를 계산
        return out 

In [6]:
model = ResNet().to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)
criterion = nn.CrossEntropyLoss()

print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

In [8]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [9]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.533777
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.931172
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.537740
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.964626
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.247007
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.348087
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.291493
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.084874

[EPOCH: 1], 	Test Loss: 0.0403, 	Test Accuracy: 54.32 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 1.212604
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 1.159376
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 1.023008
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 0.972971
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 0.817964
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 0.933632
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 0.806798
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 0.914227

[EPOCH: 2], 	Test Loss: 0.0352, 	Test Accuracy: 61.86 % 